# [xTuring](https://github.com/stochasticai/xturing)  fine-tuning bloom 模型教程


## 1. 安装  `xTuring` 及相关库

In [ ]:
!pip install xturing accelerate --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.0/97.0 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.3/215.3 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 718.6/718.6 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.3/70.3 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 57.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 764.8/764.8 kB 42.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 48.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.2/84.

## 2. 处理中文数据集





数据集的格式还是挺简单的，这里使用的和斯坦福羊驼数据集格式一致，如下：
其中
- instruction 指令，即问题
- input 输入，类似于参数，比如下面的 **编辑文章，使其更吸引读者**
- outpu 答案

对于 instruction 和 input 实际上是可以合并到一起，有些数据集就合并到一起了。

```json
[
  {
    "instruction": "我们如何在日常生活中减少用水？",
    "input": "",
    "output": "1. 使用节水装置，如节水淋浴喷头和水龙头。 \n2. 使用水箱或水桶收集家庭废水，例如洗碗和洗浴。 \n3. 在社区中提高节水意识。 \n4. 检查水管和灌溉系统的漏水情况，并及时修复它们。 \n5. 洗澡时间缩短，使用低流量淋浴头节约用水。 \n6. 收集雨水，用于园艺或其他非饮用目的。 \n7. 刷牙或擦手时关掉水龙头。 \n8. 减少浇水草坪的时间。 \n9. 尽可能多地重复使用灰水（来自洗衣机、浴室水槽和淋浴的水）。 \n10. 只购买能源效率高的洗碗机和洗衣机。"
  },
  {
    "instruction": "编辑文章，使其更吸引读者。",
    "input": "自主机器人是计算机控制的机器，被编程执行特定任务而不需要任何人类输入。自主机器人在各个行业中被越来越广泛地应用，从制造业到医疗保健再到安全。",
    "output": "自主机器人是计算机控制的机器，被编程执行特定任务而不需要任何人类输入，从而实现了新的效率、精确度和可靠性水平。自主机器人在各个行业中被越来越广泛地应用，从制造业，它们可以使用精度和一致的质量组装复杂的组件，到医疗保健，可以协助进行医疗测试和处理，再到安全，可以监控大面积地区，保障人们和财产的安全。自主机器人还可以减少在危险或有害环境中的错误和增加安全，在工业流程的检查或维修期间等。由于其多样性，自主机器人将彻底改变我们工作方式的方式，使任务变得更加简单、快速，最终更加愉悦。"
  },
  {
    "instruction": "政府可以采取哪些策略来减少空气污染？",
    "input": "",
    "output": "1. 实施强制的车辆排放标准和基于激励的计划，以降低车辆的碳足迹。\n2. 增加公共交通工具，减少公众对车辆的依赖。\n3. 增加对空气污染的影响的认识，鼓励市民减少污染物的生成。\n4. 投资于可再生能源的研究和开发，如太阳能和风能。\n5. 在工厂和发电厂安装空气污染控制装置，例如洗涤器。\n6. 对车辆和工厂使用清洁燃料。\n7. 实施更好的城市规划和控制拓展。\n8. 改善农业效率，减少化肥和杀虫剂的使用。\n9. 种植更多的树木以减少空气污染。\n10. 减少木材、煤炭和生物质的燃烧。"
  }
```

In [ ]:
!wget https://github.com/ymcui/Chinese-LLaMA-Alpaca/raw/main/data/alpaca_data_zh_51k.json
!rm -rf alpaca_data

--2023-04-18 03:56:50--  https://github.com/ymcui/Chinese-LLaMA-Alpaca/raw/main/data/alpaca_data_zh_51k.json
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/ymcui/Chinese-LLaMA-Alpaca/main/data/alpaca_data_zh_51k.json [following]
--2023-04-18 03:56:50--  https://raw.githubusercontent.com/ymcui/Chinese-LLaMA-Alpaca/main/data/alpaca_data_zh_51k.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18810090 (18M) [text/plain]
Saving to: ‘alpaca_data_zh_51k.json’

alpaca_data_zh_51k. 100%[===================>]  17.94M  --.-KB/s    in 0.04s   

2023-04-18 03:56:52 (409 MB/s) - ‘alpaca_data_zh_51k.

转换数据集，这个数据集格式有问题需要处理一下


In [ ]:
# -*- coding: utf-8 -*-

import json

from datasets import Dataset, DatasetDict

# Convert the alpaca JSON dataset to HF format

data_to_train=1000

# Right now only the HuggingFace datasets are supported, that's why the JSON Alpaca dataset
# needs to be converted to the HuggingFace format. In addition, this HF dataset should have 3 columns for instruction finetuning: instruction, text and target.
def preprocess_alpaca_json_data(alpaca_dataset_path: str):
    """Creates a dataset given the alpaca JSON dataset. You can download it here: https://raw.githubusercontent.com/tatsu-lab/stanford_alpaca/main/alpaca_data.json

    :param alpaca_dataset_path: path of the Alpaca dataset
    """
    alpaca_data = json.load(open(alpaca_dataset_path,encoding='utf-8'))
    instructions = []
    inputs = []
    outputs = []
    idx = 0
    for data in alpaca_data:
        if idx >= data_to_train:
          break
        if 'output' not in data:
          if '输出' not in data:
            continue
          data['output'] = data['输出']
        if 'input' not in data:
          data['input'] = data['输入']
        if 'instruction' not in data:
          if '指令' in data:
            data['instruction'] = data['指令']  
          elif '指示' in data:
            data['instruction'] = data['指示']      


        instructions.append(data["instruction"])
        inputs.append(data["input"])
        outputs.append(data["output"])
        idx += 1

    data_dict = {
        "train": {"instruction": instructions, "text": inputs, "target": outputs}
    }

    dataset = DatasetDict()
    # using your `Dict` object
    for k, v in data_dict.items():
        dataset[k] = Dataset.from_dict(v)

    dataset.save_to_disk(str("./alpaca_data"))

data='./alpaca_data_zh_51k.json'    

preprocess_alpaca_json_data(data)


Saving the dataset (0/1 shards):   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
# 查看数据集大小
!ls -hs alpaca_data_zh_51k.json
!ls -hs alpaca_data/train

18M alpaca_data_zh_51k.json
total 324K
316K data-00000-of-00001.arrow	4.0K dataset_info.json	4.0K state.json


## 3. 加载并初始化模型

## 自定义模型

因为这个仓库还有一个 finetuning_config.yaml，配置了模型微调的参数，估计可以通过，在自定义模型类里面修改finetuning_args，添加配置。这里就不折腾了，直接因为模型结构是一样的，只是仓库不一样，所以这里直接用"bloom_lora"的名字覆盖掉。

In [ ]:
from pathlib import Path
from typing import Optional, Union

from xturing.engines.causal import CausalEngine, CausalLoraEngine

    
class BloomZhLoraEngine(CausalLoraEngine):
    config_name: str = "bloom_lora_engine"

    def __init__(self, weights_path: Optional[Union[str, Path]] = None):
        super().__init__(
            model_name="YeungNLP/bloom-1b4-zh",
            weights_path=weights_path,
            target_modules=["query_key_value"],
        )

        self.tokenizer.pad_token = self.tokenizer.eos_token
        self.tokenizer.pad_token_id = self.tokenizer.eos_token_id

from xturing.models.causal import CausalLoraModel

class BloomZhLora(CausalLoraModel):
    config_name: str = "bloom_lora"

    def __init__(self, weights_path: Optional[str] = None):
        super().__init__(BloomZhLoraEngine.config_name, weights_path)


In [ ]:
from xturing.datasets.instruction_dataset import InstructionDataset
from xturing.models.base import BaseModel
from xturing.engines.base import BaseEngine
#注册自定义模型
del BaseModel.registry[BloomZhLora.config_name] # 避免重复执行的时候，重复注册报错
BaseModel.add_to_registry(BloomZhLora.config_name, BloomZhLora) 
del BaseEngine.registry[BloomZhLoraEngine.config_name]
BaseEngine.add_to_registry(BloomZhLoraEngine.config_name, BloomZhLoraEngine)


#加载数据集
instruction_dataset = InstructionDataset("./alpaca_data")
#初始化模型训练方案
model = BaseModel.create("bloom_lora")

trainable params: 1572864 || all params: 1304684544 || trainable%: 0.120555118647899


## 4.开始微调

In [ ]:
# 可以设置一些微调参数，这里我们使用默认的，详细可以看下载 xturing 源码后，参见 finetuning_config.yaml
# finetuning_config = model.finetuning_config()
# finetuning_config.batch_size = 64 # 是一次训练中使用的样本数，在训练过程中，每次使用一个 Batch 的数据来更新模型的参数，如果显存不够可以调小
# finetuning_config.num_train_epochs = 1 # Epoch 是一个训练周期，指的是将整个数据集训练一次的过程。一般得训练多次模型能力才能稳定
# finetuning_config.learning_rate = 1e-5 # learning rate（学习率）是控制神经网络训练过程中参数更新的步长大小的超参数。如果学习率设置过大，参数更新步长会过大，可能导致参数更新过程中跳过了损失函数的最小值，导致模型无法收敛甚至发生震荡；如果学习率设置过小，参数更新步长会过小，导致模型收敛速度缓慢，训练时间较长。
# finetuning_config.weight_decay = 0.01 # weight decay（权重衰减）是一种正则化技术，用于避免模型过拟合。它通过在损失函数中增加一个惩罚项，来惩罚模型中较大的权重值，从而使得模型更加平滑和简单，避免出现过拟合问题。
# finetuning_config.optimizer_name = "adamw" # 优化器名称(具有有哪些优化器，我也不知道哈) optimizer（优化器）是一种用于训练神经网络的算法，它决定了神经网络参数的更新方式。优化器通过最小化损失函数来调整神经网络的参数，使得神经网络可以更好地拟合训练数据。
# finetuning_config.output_dir = "training_dir/" # 模型保存路径
model.finetune(dataset=instruction_dataset)

INFO:pytorch_lightning.utilities.rank_zero:Using 16bit Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name          | Type      | Params
--------------------------------------------
0 | pytorch_model | LoraModel | 1.3 B 
--------------------------------------------
1.6 M     Trainable params
1.3 B     Non-trainable params
1.3 B     Total params
5,218.738 Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=3` reached.


生成的模型在当前目录 saved_model 文件夹下面

## 5. 使用微调后的模型生成文本


In [ ]:
# 模型训练完成后，可以开始推理，验证一下上面的三个问题吧
output = model.generate(texts=["我们如何在日常生活中减少用水？"])
print("模型生成的文本为: {}".format(output))

模型生成的文本为: ['在日常生活中，我们如何减少用水？我们可以在以下几个方面减少用水：\n\n\n\n\n\n\n\n\n\n']


很差，补全失败

In [ ]:
# 生成配置
generation_config = model.generation_config()
print(generation_config)

penalty_alpha=0.6 top_k=4 max_new_tokens=256 do_sample=False top_p=None


配置说明，直接机翻：
- max_new_tokens: 生成的最大令牌数,忽略提示中的令牌数。
- penalty_alpha: 对比搜索解码时使用。该值平衡模型置信度和退化惩罚。
- top_k: 对比搜索和采样解码方法时使用。保留top k过滤的最高概率词汇令牌数。
- do_sample: 是否使用采样。 
- top_p: 用于采样解码方法。如果设置为小于1的浮点数,则只保留概率之和等于或高于top_p的最少最有可能的令牌集合进行生成。

这些参数主要用于文本生成模型的解码过程,通过不同的方法控制生成的文本结果:
- max_new_tokens 和 top_k 是通过限制生成的令牌数和概率最高的令牌来控制。
- do_sample 和 top_p 是通过按照模型输出的概率采样来控制。 
- penalty_alpha 是通过惩罚生成的重复文本来控制。
调节这些参数的值可以生成更加丰富且与提示相关的文本结果。

In [ ]:
generation_config.penalty_alpha = 0.3
generation_config.do_sample= True
generation_config.top_p = 0.8
generation_config.max_new_tokens = 500

In [ ]:
output = model.generate(texts=["我们如何在日常生活中减少用水"])
print("模型生成的文本为: {}".format(output))

模型生成的文本为: ['量？在日常生活中，我们如何减少用水量？我们可以在以下几个方面减少用水量：\n\n\n\n\n\n\n\n\n']


还是不太行，换个问题试试

In [ ]:
#这个还行
output = model.generate(texts=["政府可以采取哪些策略来减少空气污染"])
print("模型生成的文本为: {}".format(output))

模型生成的文本为: ['？\n1. 减少汽车使用\n2. 减少使用一次性塑料制品\n3. 减少使用一次性餐具\n4. 减少使用一次性塑料袋\n5. 减少使用一次性塑料容器\n6. 减少使用一次性塑料包装\n7. 减少使用一次性塑料包装\n8. 减少使用一次性塑料包装\n9. 减少使用一次性塑料包装\n10. 减少使用一次性塑料包装\n11. 减少使用一次性塑料包装\n12. 减少使用一次性塑料包装\n13. 减少使用一次性塑料包装\n14. 减少使用一次性塑料包装\n15. 减少使用一次性塑料包装\n16. 减少使用一次性塑料包装\n17. 减少使用一次性塑料包装\n18. 减少使用一次性塑料包装\n19. 减少使用一次性塑料包装\n20. 减少使用一次性塑料包装\n21. 减少使用一次性塑料包装\n22. 减少使用一次性塑料包装\n23. 减少使用一次性塑料包装\n24. 减少使用一次性塑料包装\n25. 减少使用一次性塑料包装\n26. 减少使用一次性塑料包装\n27. 减少使用一次性塑料包装\n28. 减少使用一次性塑料包装\n29. 减少使用一次性塑料包装\n30. 减少使用一次性塑料包装\n31. 减少使用一次性塑料包装\n32. 减少使用一次性塑料包装\n33. 减少使用一次性塑料包装\n34. 减少使用一次性塑料包装\n35. 减少使用一次性塑料包装\n36. 减少使用一次性塑料包装\n37. 减少使用一次性塑料包装\n38. 减少使用一次性塑料包装\n39. 减少使用一次性塑料包装\n40. 减少使用一次性塑料包装\n41. 减少使用一次性塑料包装\n42. 减少使用一次性塑料包装\n43. 减少使用一次性塑料包装\n44. 减少使用一次性塑料包装\n45. 减少使用一次性塑料包装\n46. 减少使用一次性塑料包装\n47. 减少使用一次性塑料包装\n48. 减少使用一次性塑料包装\n49. 减少使用一次性塑料包装\n50. 减少使用一次性塑料包装\n51. 减少使用一次性塑料包装\n52. 减少使用一次性塑料包装\n53. 减少使用一次性塑料包装\n54. 减少使用一次性塑料包装\n55. 减少使用一次性塑料包装\n56. 减少使用']


In [ ]:
#把指令和输入合并，这个居然返回空了
output = model.generate(texts=["编辑文章，使其更吸引读者。自主机器人是计算机控制的机器，被编程执行特定任务而不需要任何人类输入。自主机器人在各个行业中被越来越广泛地应用，从制造业到医疗保健再到安全。"])
print("模型生成的文本为: {}".format(output))

模型生成的文本为: ['']
